In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import sys
from collections import Counter
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing
from multiprocessing import Pool
import itertools
from copy import deepcopy
import random

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, PReLU, GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical, Sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169#, preprocess_input
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.utils import multi_gpu_model
import tensorflow as tf

from datetime import datetime
import pickle
import scipy.stats as stats
import os
import sklearn.metrics

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
y = pd.read_csv("data/train.csv")
yEval = pd.read_csv("data/test.csv")
classMap = pd.read_csv("data/class_map.csv")

In [4]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [5]:
y = y.set_index("image_id")

In [6]:
tables = [pq.read_table('data/train_image_data_{0}.parquet'.format(i)) for i in range(4)]
tables = [table.to_pandas() for table in tables]
df = pd.concat(tables)
df = df.set_index("image_id")
del tables

In [7]:
size=(137,137)

def calcRotate(img):
    detector = cv2.ORB_create()
    keypoints=detector.detect(img)
    descriptors=detector.compute(img,keypoints)
    angles=[]
    weights=[]
    for i in descriptors[0]:
        if i.angle!=-1:
            angles.append(i.angle)
            weights.append(i.response)
    if len(angles)==0:
        return 0
    else:
        return np.average(angles,weights=weights)

def transformImg(img):
    ret2, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    img = 255-img
    mu = cv2.moments(img, False)
    x, y= mu["m10"]/mu["m00"] , mu["m01"]/mu["m00"]
    M = np.float32([[1,0,(236/2)-x],[0,1,(137/2)-y]])
    img = cv2.warpAffine(img,M,(img.shape[1],img.shape[0]))
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    kernel = np.ones((5,5),np.float32)/25
    #img = cv2.filter2D(img,-1,kernel)
    #img = cv2.filter2D(img,-1,kernel)
    img = cv2.resize(img, (size[0],size[1]))
    angle = calcRotate(img)
    img = rotateImg(img,angle=-1*angle,prob=False)
    #img = preprocess_input(img)
    return img

def rotateImg(img, angle=None,prob=True):
    if prob==False or np.random.random()<0.5:
        if angle==None:
            angle=np.random.random()*60-30
        trans = cv2.getRotationMatrix2D((img.shape[0]/2, img.shape[1]/2), angle , 1.0)
        img = cv2.warpAffine(img, trans, (img.shape[1],img.shape[0]))
        return img
    else:
        return img

def shiftImg(img, prob=True):
    if prob==False or np.random.random()<0.5:
        x=np.random.random()*30-15
        y=np.random.random()*30-15
        trans = np.float32([[1,0,x],[0,1,y]])
        img = cv2.warpAffine(img, trans, (img.shape[1],img.shape[0]))
        return img
    else:
        return img

def randomErase(img, prob=True):
    # random erasing
    # https://github.com/yu4u/cutout-random-erasing
    p = 0.5
    s_l = 0.02
    s_h = 0.4
    r_1 = 0.3
    r_2 = 1 / 0.3
    v_l = 0
    v_h = 255
    input_size=size[0]
    if prob==False or np.random.random()<p:

        while True:
            s = np.random.uniform(s_l, s_h) * input_size * input_size
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, input_size)
            top = np.random.randint(0, input_size)
            if left + w <= input_size and top + h <= input_size:
                break
        c = np.random.uniform(v_l, v_h, (h, w, 3))
        img[top : top + h, left : left + w, :] = c
    return img

In [8]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, idx):
        _imgs=self.X[idx * self.batch_size:(idx + 1) * self.batch_size,:,:]
        
        p=False
            
        
        imgs=[]
        for img in _imgs:
            if p:
                imgs.append(np.fft.fft2(transformImg(img)))
            else:
                imgs.append(transformImg(img))
         
        if self.training:
            imgs = [rotateImg(img) for img in imgs]
            imgs = [shiftImg(img) for img in imgs]
        
        imgs=np.asarray(imgs)
        
        
        ret_y=[]
        for label in labels:
            ret_y.append(to_categorical(self.y[idx * self.batch_size:(idx + 1) * self.batch_size][label],num_classes=len(set(y[label]))))
    
    
        #mix up
        if self.training and np.random.random()<0.8:
            r= np.random.permutation(imgs.shape[0])
            imgs2=deepcopy(imgs)[r]
            grapheme=ret_y[0]
            vowel=ret_y[1]
            consonant=ret_y[2]
            grapheme2=deepcopy(grapheme)[r]
            vowel2=deepcopy(vowel)[r]
            consonant2=deepcopy(consonant)[r]
            alpha=np.random.randn(imgs.shape[0])*0.3+0.5
            alpha[alpha>1]=1
            alpha[alpha<0]=0
            imgs=np.tile(alpha,(3,*size,1)).T*imgs+np.tile((1-alpha),(3,*size,1)).T*imgs2
            grapheme=np.tile(alpha,(168,1)).T*grapheme+np.tile((1-alpha),(168,1)).T*grapheme2
            vowel=np.tile(alpha,(11,1)).T*vowel+np.tile((1-alpha),(11,1)).T*vowel2
            consonant=np.tile(alpha,(7,1)).T*consonant+np.tile((1-alpha),(7,1)).T*consonant2
            grapheme=grapheme.astype(np.float32)
            vowel=vowel.astype(np.float32)
            consonant=consonant.astype(np.float32)
            ret_y=[grapheme,vowel,consonant]
        
            
        if self.training:
            imgs = [randomErase(img) for img in imgs]
            
        if p:
            imgs=[np.real(np.fft.ifft2(img)) for img in imgs]
            
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y



In [9]:
def getMultiXceptionBased():
    model =  Xception(weights="imagenet", include_top=False)
    x = model.output  
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.5)(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236),y,train_size=0.9,random_state=8000)
del df

In [11]:
early_stopping =  EarlyStopping(monitor='val_loss', min_delta=0.0, patience=20)
checkpoint = ModelCheckpoint(filepath="tmp-Xception-epoch{epoch:04}.h5")

In [ ]:
train_gen = DataLoader(X_train, y_train, training=True, batch_size=64)
valid_gen = DataLoader(X_test, y_test, training=False, batch_size=64)
model = getMultiXceptionBased()
model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[2.,1.,1.])
print("compiled")
model.fit_generator(train_gen, validation_data=valid_gen, epochs=60, callbacks=[early_stopping, checkpoint], workers=multiprocessing.cpu_count(), use_multiprocessing=True)
model.save("multiXception.h5")

compiled
Epoch 1/60
2824/2825 [============================>.] - ETA: 0s - loss: 7.7730 - dense_1_loss: 2.9406 - dense_2_loss: 1.1671 - dense_3_loss: 0.7247 - dense_1_acc: 0.4293 - dense_2_acc: 0.7012 - dense_3_acc: 0.8000Epoch 1/60
Epoch 1/60
2825/2825 [==============================] - 977s 346ms/step - loss: 7.7713 - dense_1_loss: 2.9400 - dense_2_loss: 1.1669 - dense_3_loss: 0.7246 - dense_1_acc: 0.4294 - dense_2_acc: 0.7013 - dense_3_acc: 0.8001 - val_loss: 1.6493 - val_dense_1_loss: 0.7845 - val_dense_2_loss: 0.2401 - val_dense_3_loss: 0.2043 - val_dense_1_acc: 0.7957 - val_dense_2_acc: 0.9305 - val_dense_3_acc: 0.9378
Epoch 2/60
2825/2825 [==============================] - 941s 333ms/step - loss: 5.8886 - dense_1_loss: 2.1916 - dense_2_loss: 0.9271 - dense_3_loss: 0.5783 - dense_1_acc: 0.6449 - dense_2_acc: 0.8135 - dense_3_acc: 0.8651 - val_loss: 1.2615 - val_dense_1_loss: 0.5855 - val_dense_2_loss: 0.1704 - val_dense_3_loss: 0.1416 - val_dense_1_acc: 0.8381 - val_dense_2_acc: 